# ETL Development
### phase 0
| Task                                  | Status      | Comments  |
|---------------------------------------|-------------|-----------|
| Manually clone Azure repo             | Complete    | use shallow clone <br>* still receiving 600k+ objects (3.57GiB) <br>* full clone was 7.9 million      |
| Crawl through repo & get all md files | In Progress | None      |
| Prepare md for indexing               | In progress | None      |

In [ ]:
# Set global vars
import os

PROJECT_ROOT = "/Users/alex/Desktop/code_projects/commuter-copilot/"
RAW_DATA_SOURCE_PATH = os.path.join(
    PROJECT_ROOT, "local_files", "data", "raw", "azure-docs"
)
STAGED_DATA_SOURCE_PATH = os.path.join(
    PROJECT_ROOT, "local_files", "data", "staging", "azure-docs-staging"
)

# Ensure the directories exist
os.makedirs(RAW_DATA_SOURCE_PATH, exist_ok=True)
os.makedirs(STAGED_DATA_SOURCE_PATH, exist_ok=True)

In [ ]:
# manually update & clone azure repo to local_files/ in terminal:
!bash -c "$PROJECT_ROOT/data/etl/run_etl_azure_docs_raw.sh"